In [3]:
# !pip install -r /teamspace/studios/this_studio/eda-bids-hackathon-prep/sentinel2-modelling/requirements.txt

In [4]:
import os
import tempfile
from typing import Dict, Optional
from glob import glob

import torch
import torch.nn as nn
import torchvision.transforms as T

from PIL import Image
from torch import Tensor
from torch.utils.data import DataLoader

from torchgeo.datasets import OSCD
from torchgeo.datamodules import OSCDDataModule
from torchgeo.trainers import SemanticSegmentationTask
from torchgeo.datasets.utils import unbind_samples

from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.loggers import WandbLogger, TensorBoardLogger

import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision.transforms import Compose
from tqdm import tqdm

from sklearn.metrics import precision_score, recall_score

seed_everything(543)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

Seed set to 543


cpu


In [16]:
BANDS = "all" # or all
if BANDS == "rgb":
    CHANNELS = 6
elif BANDS == "all":
    CHANNELS = 26

In [17]:
datamodule = OSCDDataModule(
    batch_size=128,
    num_workers=8,
    download=True,
    bands=BANDS,
    patch_size=64
)

datamodule.prepare_data()

Viz a sample, remembering they are patchified on training

In [18]:
# datamodule.setup(stage="fit")
# for i in range(len(datamodule.train_dataset)):
#     fig = datamodule.train_dataset.dataset.plot(datamodule.train_dataset[i])

See https://github.com/microsoft/torchgeo/pull/1656 for casting data

In [19]:
task = SemanticSegmentationTask(
    model="unet",
    backbone="resnet18",
    weights=True,
    in_channels=CHANNELS,
    num_classes=2,
    loss="ce",
    ignore_index=None,
)

tb_logger = TensorBoardLogger(
    save_dir="tb_logs/",
)

In [20]:
trainer = Trainer(
    logger=[tb_logger],
    min_epochs=100,
    max_epochs=250,
)

_ = trainer.fit(model=task, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | CrossEntropyLoss | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | test_metrics  | MetricCollection | 0     
4 | model         | Unet             | 14.4 M
---------------------------------------------------
14.4 M    Trainable params
0         Non-trainable params
14.4 M    Total params
57.602    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:281: PossibleUserWarning: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [27]:
trainer.test(model=task, datamodule=datamodule)

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EarlyStopping, ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   test_MulticlassAccuracy   │     0.9501640200614929      │
│ test_MulticlassJaccardIndex │     0.9050595164299011      │
│          test_loss          │     0.19129081070423126     │
└─────────────────────────────┴─────────────────────────────┘

[{'test_loss': 0.19129081070423126,
  'test_MulticlassAccuracy': 0.9501640200614929,
  'test_MulticlassJaccardIndex': 0.9050595164299011}]

RGB metrics:
```
[{'test_loss': 0.19295702874660492,
  'test_MulticlassAccuracy': 0.9477683901786804,
  'test_MulticlassJaccardIndex': 0.9007222652435303}]
```

All bands metrics:
```
[{'test_loss': 0.17245826125144958,
  'test_MulticlassAccuracy': 0.9571273922920227,
  'test_MulticlassJaccardIndex': 0.9177797436714172}]
  ```

# Compute metrics

In [28]:
model = task.model.to(device).eval()

In [29]:
datamodule.setup(stage="fit")

In [30]:
batch = next(iter(datamodule.test_dataloader()))
batch = datamodule.aug(batch)
batch.keys()

dict_keys(['image', 'mask'])

In [31]:
y_preds = []
y_trues = []
for batch in tqdm(datamodule.test_dataloader()):
    batch = datamodule.aug(batch)
    images = batch["image"].to(device)
    y_trues.append(batch["mask"].numpy())
    with torch.inference_mode():
        y_pred = model(images).argmax(dim=1).cpu().numpy()
    y_preds.append(y_pred)
y_preds = np.concatenate(y_preds, axis=0).reshape(-1)
y_trues = np.concatenate(y_trues, axis=0).reshape(-1)

100%|██████████| 10/10 [00:04<00:00,  2.22it/s]


In [32]:
precision = precision_score(y_trues[::50], y_preds[::50])
recall = recall_score(y_trues[::50], y_preds[::50])
f1 = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

Precision: 0.4657, Recall: 0.2403, F1: 0.3171


RGB metrics: Precision: 0.2883, Recall: 0.1020, F1: 0.1507

All bands metrics: Precision: 0.4189, Recall: 0.2082, F1: 0.2782